In [1]:
import random
import pandas as pd
import numpy as np
import os
import cv2
from PIL import Image
import util

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
from torchvision import transforms as T
from torchsummary import summary

from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score

import util
import warnings
warnings.filterwarnings(action='ignore')

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
util.seed_everything(41) # Seed 고정

In [2]:
CFG = {
    'IMG_SIZE_TRAIN':380,
    'IMG_SIZE_TEST':380,
    'EPOCHS':10,
    'LEARNING_RATE':1e-3,
    'LEARNING_RATE_DECREASE': 0.1,
    'MIN_LEARNING_RATE': 1e-5,
    'BATCH_SIZE':20,
    'SEED':41,
    'MODEL_NAME':"v2-s",
    'LABEL_SMOOTH': 0.1
}

In [3]:
if CFG["MODEL_NAME"] == "b0":
    class BaseModel(nn.Module):
        def __init__(self, num_classes=10):
            super(BaseModel, self).__init__()
            self.backbone = models.efficientnet_b0(pretrained=True)
            self.backbone.classifier[-1] = nn.Linear(1280, num_classes)
            
        def forward(self, x):
            x = self.backbone(x)
            x = F.sigmoid(x)
            return x
            
elif CFG["MODEL_NAME"] == "b4":
    class BaseModel(nn.Module):
        def __init__(self, num_classes=10):
            super(BaseModel, self).__init__()
            self.backbone = models.efficientnet_b4(pretrained=True)
            self.backbone.classifier[-2] = nn.Dropout(0.7)
            self.backbone.classifier[-1] = nn.Linear(1792, num_classes)
            
        def forward(self, x):
            x = self.backbone(x)
            x = F.sigmoid(x)
            return x
            
elif CFG["MODEL_NAME"] == "b7":
    class BaseModel(nn.Module):
        def __init__(self, num_classes=10):
            super(BaseModel, self).__init__()
            self.backbone = models.efficientnet_b7(pretrained=True)
            self.backbone.classifier[-1] = nn.Linear(2560, num_classes)
            
        def forward(self, x):
            x = self.backbone(x)
            x = F.sigmoid(x)
            return x

elif CFG["MODEL_NAME"] == "v2-s":
    class BaseModel(nn.Module):
        def __init__(self, num_classes=10):
            super(BaseModel, self).__init__()
            self.backbone = models.efficientnet_v2_s(pretrained=True).features
            self.adaptiveavgpool = nn.AdaptiveAvgPool2d(output_size=(1,1))
            self.classifier = nn.Sequential(nn.Flatten(),
                                            nn.Dropout(0.7), 
                                            nn.Linear(1280,10))
            
        def forward(self, x):
            x = self.backbone(x)
            x = self.adaptiveavgpool(x)
            x = self.classifier(x)
            x = F.sigmoid(x)
            return x

elif CFG["MODEL_NAME"] == "v2-m":
    class BaseModel(nn.Module):
        def __init__(self, num_classes=10):
            super(BaseModel, self).__init__()
            self.backbone = models.efficientnet_v2_m(pretrained=True)
            self.backbone.classifier[-2] = nn.Dropout(0.7)
            self.backbone.classifier[-1] = nn.Linear(1280, num_classes)
            
        def forward(self, x):
            x = self.backbone(x)
            x = F.sigmoid(x)
            return x

In [4]:
train_labels = []
df = pd.DataFrame()
for csv in ['train.csv', 'train_mixup.csv', 'train_rmbg.csv']:
    df_ = pd.read_csv(csv, index_col="id")
    df = pd.concat([df,df_])

df = df.sample(frac=1)
train_len = int(len(df) * 0.8)
df = df.loc[:,["img_path", "A", "B", "C", "D", "E", "F", "G", "H", "I", "J"]]

train_df = df[:train_len]
val_df = df[train_len:train_len+1]

train_labels = util.get_labels(train_df)
val_labels = util.get_labels(val_df)

In [5]:
def get_train_transform(img_res= 380):
    return T.Compose([
                    T.Resize(img_res),
                    T.TrivialAugmentWide(),
                    T.AugMix(),
                    T.RandomPerspective(distortion_scale=0.6, p=0.3),
                    T.ColorJitter(brightness=.5, hue=.3),
                    T.ToTensor(),
                    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
                    ])

def get_test_transform(img_res=380):
    return T.Compose([
                    T.Resize(img_res),
                    T.ToTensor(),
                    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
                    ])
                            
train_dataset = util.CustomDataset(train_df['img_path'].values, train_labels, get_train_transform(CFG['IMG_SIZE_TRAIN']))
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=8)

val_dataset = util.CustomDataset(val_df['img_path'].values, val_labels, get_test_transform(CFG['IMG_SIZE_TEST']))
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=8)

In [6]:
model = BaseModel()
model.eval()

labeled = pd.read_csv("C:/Users/abc/Desktop/labeled.csv", index_col=0)
labeled = np.array(labeled)

for epochs in range(1, CFG['EPOCHS']+1):
    save_model_dir = "./models/Effinet-{0} epochs{1} res{2}.pth".format(CFG["MODEL_NAME"], epochs, CFG['IMG_SIZE_TRAIN'])
    lr = CFG['LEARNING_RATE'] * (CFG['LEARNING_RATE_DECREASE'] ** (epochs - 1))
    lr = max(lr, CFG['MIN_LEARNING_RATE'])
    
    if os.path.isfile(save_model_dir):
        model.load_state_dict(torch.load(save_model_dir))
        print(save_model_dir, 'loaded')
        continue

    print("epochs:", epochs, "learning_rate:", lr)

    optimizer = torch.optim.Adam(params = model.parameters(), lr = lr)
    infer_model, val_acc = util.train(model=model, optimizer=optimizer, epochs=1, train_loader=train_loader, val_loader=val_loader, scheduler=None, device=device, label_smooth=CFG['LABEL_SMOOTH'])
    torch.save(model.state_dict(), save_model_dir)

    test = pd.read_csv('./test.csv')
    test_dataset = util.CustomDataset(test['img_path'].values, None, get_test_transform(CFG['IMG_SIZE_TEST']))
    test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=4)

    pred = util.inference(model, test_loader, device)
    pred = np.array(pred > 0.5, dtype=np.int8)
    
    submission = pd.read_csv('./sample_submission.csv', index_col=0)
    submission.iloc[:,:] = pred
    submission.to_csv(save_model_dir.replace('.pth', '.csv'))

epochs: 1 learning_rate: 0.001


  0%|          | 0/26777 [00:00<?, ?it/s]

train_loss: 0.47685

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch [1], Train Loss : [0.47685] Val Loss : [0.45766] Val ACC : [0.80000]


  0%|          | 0/73 [00:00<?, ?it/s]

380 0.9593437945791726


  0%|          | 0/73 [00:00<?, ?it/s]

380 0.9593437945791726
epochs: 2 learning_rate: 0.0001


  0%|          | 0/26777 [00:00<?, ?it/s]